<a href="https://colab.research.google.com/github/dgalassi99/OOP_learning/blob/main/ipynbs/03_Py_OOP_PortfolioManager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OOP: Building a Portfolio Manager Simulator

In the quest of learning more about quant and financial engineering I realized I am missing a bit of skills in OOP. After following a couple of lectures I think is time to get the hands dirty and try out to learn by doing.

To stay linked to the financial sector I will try to do some projects with OOP but with the economic-financial "meaning".

## Creating the Classes

In [97]:
import numpy as np
import pandas as pd
class Asset:
  def __init__(self, ticker, prices):
    self.ticker = ticker
    self.prices = prices #This is a dict

  def get_price(self, date):
    return self.prices.get(date)


Now we create the Portfolio class which will contain info such as cash, value ...

In [112]:
class Portfolio:
    def __init__(self, cash=100000):
        self.cash = cash
        self.holdings = {}
        self.transactions = []
        self.portfolio_history = []

    def buy_asset(self, asset, date, quantity):
        price = asset.get_price(date)
        cost = price * quantity
        if cost > self.cash:
            raise ValueError("Not enough cash")
        if asset in self.holdings:
            self.holdings[asset] += quantity
        else:
            self.holdings[asset] = quantity
        self.cash -= cost
        # Calculate holding value and include in transaction log
        holding_value = self.calculate_holding_value(date)
        self.transactions.append({'type': 'BUY', 'asset': asset.ticker,
                                  'date': date, 'quantity': quantity, 'price': price})
        self.update_portfolio_history(date)

    def sell_asset(self, asset, date, quantity):
        price = asset.get_price(date)
        cost = price * quantity
        if asset not in self.holdings:
            raise ValueError("Asset not in holdings")
        if self.holdings[asset] < quantity:
            raise ValueError("Not enough assets")
        self.holdings[asset] -= quantity
        self.cash += cost
        # Calculate holding value and include in transaction log
        holding_value = self.calculate_holding_value(date)
        self.transactions.append({'type': 'SELL', 'asset': asset.ticker,
                                  'date': date, 'quantity': quantity, 'price': price})
        self.update_portfolio_history(date)


    def value(self, date):
        total = self.cash
        for asset, quantity in self.holdings.items():
            total += asset.get_price(date) * quantity
        return total

    def get_transactions(self):
        return self.transactions

    def update_portfolio_history(self, date):
        """Updates the portfolio history log with current holdings, cash, and total value."""
        holding_value = self.calculate_holding_value(date)
        total_value = self.cash + holding_value

        # Modify this part to store only the quantity:
        holdings_log = {}
        for asset, quantity in self.holdings.items():
            holdings_log[asset.ticker] = quantity  # Store ticker and quantity

        self.portfolio_history.append({
            'date': date,
            'cash': self.cash,
            'holding_value': holding_value,
            'total_value': total_value,
            'holdings': holdings_log  # Store the modified holdings_log
        })

    def calculate_holding_value(self, date):
        """Calculates the total value of holdings for a given date."""
        total_holding_value = 0
        for asset, quantity in self.holdings.items():
            total_holding_value += asset.get_price(date) * quantity
        return total_holding_value

In [146]:
#check if the math works - we buy and sell at the same prices to see if tot value is constant
aapl_prices = {'2020-01-07':103.5,
              '2020-03-07':101.2,
              '2020-03-18':99.8,
              '2020-07-07':97.5,
              '2020-11-07':97.2,}
btc_prices = {'2020-01-07':10123,
              '2020-03-07':6544,
              '2020-03-18':13445,
              '2020-07-07':5321,
              '2020-11-07':15678,}

aapl = Asset('AAPL', aapl_prices)
btc = Asset('BTC', btc_prices)

pf = Portfolio(cash = 10000)
for date, price in aapl_prices.items():
    aapl_prices[date] = price
    pf.buy_asset(aapl, date, 10)
    pf.sell_asset(aapl, date, 10)

In [147]:
pd.DataFrame(pf.get_transactions())

,type,asset,date,quantity,price
0,BUY,AAPL,2020-01-07,10,103.5
1,SELL,AAPL,2020-01-07,10,103.5
2,BUY,AAPL,2020-03-07,10,101.2
3,SELL,AAPL,2020-03-07,10,101.2
4,BUY,AAPL,2020-03-18,10,99.8
5,SELL,AAPL,2020-03-18,10,99.8
6,BUY,AAPL,2020-07-07,10,97.5
7,SELL,AAPL,2020-07-07,10,97.5
8,BUY,AAPL,2020-11-07,10,97.2
9,SELL,AAPL,2020-11-07,10,97.2


In [148]:
for date, price in btc_prices.items():
    btc_prices[date] = price
    pf.buy_asset(btc, date, 0.1)
    pf.sell_asset(btc, date, 0.1)

In [149]:
pd.DataFrame(pf.get_transactions())

,type,asset,date,quantity,price
0,BUY,AAPL,2020-01-07,10.0,103.5
1,SELL,AAPL,2020-01-07,10.0,103.5
2,BUY,AAPL,2020-03-07,10.0,101.2
3,SELL,AAPL,2020-03-07,10.0,101.2
4,BUY,AAPL,2020-03-18,10.0,99.8
5,SELL,AAPL,2020-03-18,10.0,99.8
6,BUY,AAPL,2020-07-07,10.0,97.5
7,SELL,AAPL,2020-07-07,10.0,97.5
8,BUY,AAPL,2020-11-07,10.0,97.2
9,SELL,AAPL,2020-11-07,10.0,97.2


In [150]:
pd.DataFrame(pf.portfolio_history)

,date,cash,holding_value,total_value,holdings
0,2020-01-07,8965.0,1035.0,10000.0,{'AAPL': 10}
1,2020-01-07,10000.0,0.0,10000.0,{'AAPL': 0}
2,2020-03-07,8988.0,1012.0,10000.0,{'AAPL': 10}
3,2020-03-07,10000.0,0.0,10000.0,{'AAPL': 0}
4,2020-03-18,9002.0,998.0,10000.0,{'AAPL': 10}
5,2020-03-18,10000.0,0.0,10000.0,{'AAPL': 0}
6,2020-07-07,9025.0,975.0,10000.0,{'AAPL': 10}
7,2020-07-07,10000.0,0.0,10000.0,{'AAPL': 0}
8,2020-11-07,9028.0,972.0,10000.0,{'AAPL': 10}
9,2020-11-07,10000.0,0.0,10000.0,{'AAPL': 0}


In [152]:
#we also need to check what happens if i buy more times before selling and the holding cumulates
pf.buy_asset(aapl, '2020-07-07', 20)
pf.buy_asset(btc, '2020-07-07', 0.5)
pd.DataFrame(pf.portfolio_history)

,date,cash,holding_value,total_value,holdings
0,2020-01-07,8965.0,1035.0,10000.0,{'AAPL': 10}
1,2020-01-07,10000.0,0.0,10000.0,{'AAPL': 0}
2,2020-03-07,8988.0,1012.0,10000.0,{'AAPL': 10}
3,2020-03-07,10000.0,0.0,10000.0,{'AAPL': 0}
4,2020-03-18,9002.0,998.0,10000.0,{'AAPL': 10}
5,2020-03-18,10000.0,0.0,10000.0,{'AAPL': 0}
6,2020-07-07,9025.0,975.0,10000.0,{'AAPL': 10}
7,2020-07-07,10000.0,0.0,10000.0,{'AAPL': 0}
8,2020-11-07,9028.0,972.0,10000.0,{'AAPL': 10}
9,2020-11-07,10000.0,0.0,10000.0,{'AAPL': 0}


In [99]:
# check if it does not let me sell something i dont have
pf.sell_asset(aapl, '2020-01-07', 100)

ValueError: Not enough assets

In [100]:
#check if it does not let me buy if cost > price
pf.buy_asset(aapl, '2020-01-07', 1000)

ValueError: Not enough cash

## Comments

This is a very simple usega of classes i practiced with, this can be couple with strategy that buys/sells based on some logic and the portfolio manager keeps track of all the transacitions and the status of the portfolio!